#### 분류 평가
- TP, FP, FN, TN는 예측 클래스와 실제 클래스의 Positive 결정 값과 Negative 결정 값의 결합에 따라 결정
- 앞문자 True, False는 예측값과 실제 값이 같은가 틀린가를 의미하고 뒤 문자 Negative/Positive는 예측 결과 값이 부정 긍정을 의미
- TN는 예측값을 Negative 값 0으로 예측했고 실제값 역시 Negative 값 이
- FP는 예측값을 Positive 값 1로 예측했고 실제 값은 Negative 값 이
- FN은 예측값을 Negative 값 0으로 예측했고 실제 값은 Positive 값 1
- TN는 예측값을 Positive 값 1로 예측했고 실제값 역시 Positive 값 1
- 정확도 = (TP + TN) / ( TP + TN + FP + FN)
- 정밀도 = TP / ( TP + FP)
- 재현율 = TPI (TP + FN) : 정밀도와 재현율이 어느 한쪽으로 치우치지 않는 수치를 나타낼때 높아짐.
- F1 = 2 * ( 정밀도 * 재현을) / (정밀도 + 재현율)
- 정밀도와 재현율은 Positive 데이터 세트의 예측 성능에 좀 더 초점을 맞춘 평가 지표
- 재현율이 중요 지표인 경우는 실제 Positive 양성 데이터를 Negative로 잘못 판단하게 되면 업무상 큰 영향이 발생하는 경우(ex. 보험사기, 암진단)

In [3]:
titanic = pd.read_csv('../Data/train.csv')

In [5]:
titanic

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd 

df= pd.read_csv('../Data/train.csv')
df.keys()

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [18]:
y_df = df['Survived']
X_df= df.drop('Survived',axis=1)
X_df= transform_features(X_df)

X_train, X_test, y_train, y_test= train_test_split(X_df,y_df,
                                                  test_size=0.2,random_state=11)

In [20]:
lr_clf = LogisticRegression()
lr_clf.fit(X_train,y_train)
pred = lr_clf.predict(y_teest.png)
accuracy_lr = accuracy_score(y_test,pred)
accurqacy_lir


NameError: name 'y_teest' is not defined